In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
%%writefile run_combined_inference.py

import subprocess
import sys
import os

# ==============================================================================
# --- 1. SETUP: INSTALL DEPENDENCIES ---
# ==============================================================================

def install_packages():
    """Checks for and installs required Python packages."""
    print("--- Checking and installing required packages ---")
    # Add all packages your models depend on.
    required_packages = ['pandas', 'numpy', 'scipy', 'tqdm', 'joblib', 'catboost', 'mne', 'scikit-learn']
    for package in required_packages:
        try:
            __import__(package)
        except ImportError:
            print(f"Installing {package}...")
            # Use -q for a quieter installation
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
    print("--- All packages are ready ---")

# Execute the installation check before doing anything else.
install_packages()


# ==============================================================================
# --- 2. MAIN SCRIPT LOGIC (AFTER INSTALLATION) ---
# ==============================================================================

import pandas as pd
import numpy as np
import joblib
import time
import warnings
from scipy.signal import butter, filtfilt, welch
from tqdm import tqdm
from catboost import CatBoostClassifier
from mne.decoding import CSP

# --- Configuration ---
# These paths are set up for a typical Google Drive structure.
# Adjust them if your folder layout is different.
DRIVE_BASE_PATH = '/content/drive/MyDrive/aic3'
RAW_DATA_PATH = os.path.join(DRIVE_BASE_PATH, 'unzipped')
ARTIFACTS_BASE_PATH = '/content/drive/MyDrive/aic3/ssvep_model_artifacts'
MI_ARTIFACTS_PATH = os.path.join(DRIVE_BASE_PATH, 'MI')
SSVEP_ARTIFACTS_PATH = os.path.join(ARTIFACTS_BASE_PATH)

SUBMISSION_FILE_PATH = os.path.join(DRIVE_BASE_PATH, 'combined_submission.csv')
TEST_CSV_PATH = os.path.join(RAW_DATA_PATH, 'test.csv')

# --- MI Task Constants ---
MI_CHANNELS = ['C3', 'CZ', 'C4']
MI_TRIAL_SAMPLES = 2250

# --- SSVEP Task Constants ---
SSVEP_CHANNELS = ['PO7', 'PZ', 'OZ', 'PO8']
SSVEP_SAMPLING_RATE = 250
SSVEP_TRIAL_DURATION = 7.0
SSVEP_TRIM_START_S = 1.0
SSVEP_TARGET_FREQS = [7, 8, 10, 13]

warnings.filterwarnings('ignore')

# ==============================================================================
# --- MOTOR IMAGERY (MI) HELPER FUNCTIONS ---
# ==============================================================================

def load_mi_test_trial(row, base_path):
    path = os.path.join(base_path, 'MI', 'test', row['subject_id'], str(row['trial_session']), 'EEGdata.csv')
    eeg_data = pd.read_csv(path)
    start_idx = (int(row['trial']) - 1) * MI_TRIAL_SAMPLES
    end_idx = start_idx + MI_TRIAL_SAMPLES
    return eeg_data.iloc[start_idx:end_idx][MI_CHANNELS].values

def mi_normalize_trial(trial_data):
    mean = trial_data.mean(axis=0, keepdims=True)
    std = trial_data.std(axis=0, keepdims=True)
    return (trial_data - mean) / (std + 1e-8)

def mi_bandpass_filter(data, low=8, high=30, fs=250, order=4):
    nyquist = 0.5 * fs
    b, a = butter(order, [low / nyquist, high / nyquist], btype='band')
    return filtfilt(b, a, data, axis=0)

def mi_apply_subject_norm(trial_data, subject_id, subject_stats):
    stats = subject_stats['subjects'].get(subject_id, subject_stats['global'])
    return (trial_data - stats['mean']) / (stats['std'] + 1e-8)

def mi_compute_psd(trial, fs=250, bands=[(0.5,4), (4,8), (8,13), (13,30), (30,50)]):
    features = []
    for ch in range(trial.shape[1]):
        f, Pxx = welch(trial[:, ch], fs=fs, nperseg=256)
        for band in bands:
            fmin, fmax = band
            features.append(np.trapz(Pxx[(f >= fmin) & (f <= fmax)], f[(f >= fmin) & (f <= fmax)]))
    return np.array(features)

def predict_mi_trial(row, artifacts, base_data_path):
    trial_data = load_mi_test_trial(row, base_data_path)
    normalized_data = mi_normalize_trial(trial_data)
    filtered_data = mi_bandpass_filter(normalized_data)
    final_preprocessed_data = mi_apply_subject_norm(filtered_data, row['subject_id'], artifacts['stats'])
    csp_input = final_preprocessed_data.T.reshape(1, len(MI_CHANNELS), MI_TRIAL_SAMPLES)
    csp_features = artifacts['csp'].transform(csp_input)
    psd_features = mi_compute_psd(final_preprocessed_data).reshape(1, -1)
    combined_features = np.hstack([csp_features, psd_features])
    prediction_encoded = artifacts['model'].predict(combined_features)
    return 'Left' if int(prediction_encoded[0]) == 0 else 'Right'

# ==============================================================================
# --- SSVEP HELPER FUNCTIONS ---
# ==============================================================================

def load_ssvep_test_trial(row, base_path):
    path = os.path.join(base_path, 'SSVEP', 'test', row['subject_id'], str(row['trial_session']), 'EEGdata.csv')
    eeg_data = pd.read_csv(path).ffill().bfill()
    samples_per_trial = int(SSVEP_TRIAL_DURATION * SSVEP_SAMPLING_RATE)
    trim_samples = int(SSVEP_TRIM_START_S * SSVEP_SAMPLING_RATE)
    start_idx = (int(row['trial']) - 1) * samples_per_trial
    return eeg_data.iloc[start_idx + trim_samples : start_idx + samples_per_trial][SSVEP_CHANNELS].values

def ssvep_bandpass_filter(data):
    low, high, order = 4.0, 42.0, 4
    nyq = 0.5 * SSVEP_SAMPLING_RATE
    b, a = butter(order, [low / nyq, high / nyq], btype='band')
    return np.apply_along_axis(lambda x: filtfilt(b, a, x), 0, data)

def ssvep_get_psd_features(data):
    all_features = []
    nperseg = SSVEP_SAMPLING_RATE * 4
    for i in range(data.shape[1]):
        freqs, psd = welch(data[:, i], fs=SSVEP_SAMPLING_RATE, nperseg=nperseg, nfft=nperseg * 2)
        for f in SSVEP_TARGET_FREQS:
            for h in range(1, 4):
                harmonic_freq = h * f
                target_idx = np.argmin(np.abs(freqs - harmonic_freq))
                noise_indices = np.where((freqs >= harmonic_freq - 3) & (freqs <= harmonic_freq + 3) & (np.abs(freqs - harmonic_freq) > 0.25))[0]
                signal_power = psd[target_idx]
                noise_power = np.mean(psd[noise_indices]) if len(noise_indices) > 0 else 1e-12
                all_features.extend([signal_power, signal_power / noise_power])
    return np.array(all_features)

def predict_ssvep_trial(row, artifacts, base_data_path):
    eeg_signals = load_ssvep_test_trial(row, base_data_path)
    filtered_signals = ssvep_bandpass_filter(eeg_signals)
    features = ssvep_get_psd_features(filtered_signals).reshape(1, -1)
    scaled_features = artifacts['scaler'].transform(features)
    selected_features = artifacts['selector'].transform(scaled_features)
    prediction_encoded = artifacts['model'].predict(selected_features)
    return artifacts['le'].inverse_transform(prediction_encoded)[0]

# ==============================================================================
# --- MAIN INFERENCE ORCHESTRATOR ---
# ==============================================================================

def main():
    """Main function to load all artifacts and run inference for all tasks."""
    print("--- [1/5] Starting COMBINED INFERENCE Pipeline ---")

    # --- Load MI Artifacts ---
    print(f"--- [2/5] Loading MI artifacts from: {MI_ARTIFACTS_PATH} ---")
    try:
        mi_artifacts = {
            'model': joblib.load(os.path.join(MI_ARTIFACTS_PATH, 'catboost_mi_model.pkl')),
            'csp': joblib.load(os.path.join(MI_ARTIFACTS_PATH, 'csp_mi_transformer.pkl')),
            'stats': joblib.load(os.path.join(MI_ARTIFACTS_PATH, 'mi_subject_stats.pkl'))
        }
        print("MI artifacts loaded successfully.")
    except Exception as e:
        print(f"FATAL ERROR: Could not load MI artifacts. {e}\nCheck that the path is correct and the files exist.")
        return

    # --- Load SSVEP Artifacts ---
    print(f"--- [3/5] Loading SSVEP artifacts from: {SSVEP_ARTIFACTS_PATH} ---")
    try:
        ssvep_artifacts = {
            'model': joblib.load(os.path.join(SSVEP_ARTIFACTS_PATH, 'ssvep_model.joblib')),
            'scaler': joblib.load(os.path.join(SSVEP_ARTIFACTS_PATH, 'ssvep_scaler.joblib')),
            'selector': joblib.load(os.path.join(SSVEP_ARTIFACTS_PATH, 'ssvep_selector.joblib')),
            'le': joblib.load(os.path.join(SSVEP_ARTIFACTS_PATH, 'ssvep_label_encoder.joblib'))
        }
        print("SSVEP artifacts loaded successfully.")
    except Exception as e:
        print(f"FATAL ERROR: Could not load SSVEP artifacts. {e}\nCheck that the path is correct and the files exist.")
        return

    # --- Load Test Metadata ---
    print(f"--- [4/5] Loading test metadata from: {TEST_CSV_PATH} ---")
    if not os.path.exists(TEST_CSV_PATH):
        print(f"FATAL ERROR: Test metadata file not found at {TEST_CSV_PATH}")
        return
    test_df = pd.read_csv(TEST_CSV_PATH)
    print(f"Found {len(test_df)} total test samples.")

    # --- Main Prediction Loop ---
    print("--- [5/5] Generating predictions for all tasks ---")
    results = []
    prediction_start_time = time.time()

    for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Predicting All Tasks"):
        try:
            if row['task'] == 'MI':
                label = predict_mi_trial(row, mi_artifacts, RAW_DATA_PATH)
            elif row['task'] == 'SSVEP':
                label = predict_ssvep_trial(row, ssvep_artifacts, RAW_DATA_PATH)
            else:
                label = "Error_Unknown_Task"
            results.append({'id': row['id'], 'label': label})
        except Exception as e:
            print(f"Error processing trial ID {row['id']} (Task: {row['task']}): {e}")
            results.append({'id': row['id'], 'label': 'Error_Processing_Failed'})

    prediction_end_time = time.time()

    # --- Final Step: Save Submission and Report Timings ---
    submission_df = pd.DataFrame(results).sort_values(by='id').reset_index(drop=True)
    submission_df.to_csv(SUBMISSION_FILE_PATH, index=False)

    total_prediction_time = prediction_end_time - prediction_start_time

    print("\n" + "="*50)
    print("      COMBINED INFERENCE COMPLETE")
    print("="*50)
    print(f"Total time for prediction loop: {total_prediction_time:.2f} seconds")
    print(f"Average time per sample: {total_prediction_time / len(test_df):.4f} seconds")
    print(f"\nFinal submission file saved to: {SUBMISSION_FILE_PATH}")
    print("\nSubmission Head:")
    print(submission_df.head())
    print("\nSubmission Tail:")
    print(submission_df.tail())
    print("="*50)

if __name__ == '__main__':
    main()

Overwriting run_combined_inference.py


In [13]:
# In a notebook cell
!python run_combined_inference.py

--- Checking and installing required packages ---
Installing scikit-learn...
--- All packages are ready ---
--- [1/5] Starting COMBINED INFERENCE Pipeline ---
--- [2/5] Loading MI artifacts from: /content/drive/MyDrive/aic3/MI ---
MI artifacts loaded successfully.
--- [3/5] Loading SSVEP artifacts from: /content/drive/MyDrive/aic3/ssvep_model_artifacts ---
SSVEP artifacts loaded successfully.
--- [4/5] Loading test metadata from: /content/drive/MyDrive/aic3/unzipped/test.csv ---
Found 100 total test samples.
--- [5/5] Generating predictions for all tasks ---
Predicting All Tasks: 100% 100/100 [00:26<00:00,  3.82it/s]

      COMBINED INFERENCE COMPLETE
Total time for prediction loop: 26.17 seconds
Average time per sample: 0.2617 seconds

Final submission file saved to: /content/drive/MyDrive/aic3/combined_submission.csv

Submission Head:
     id label
0  4901  Left
1  4902  Left
2  4903  Left
3  4904  Left
4  4905  Left

Submission Tail:
      id     label
95  4996     Right
96  4997   